# Capstone Project - The Battle of Neighborhoods (Week 1)

### **Question 1:** 
   Clearly define a problem or an idea of your choice, where you would need to leverage the Foursquare location data to solve or execute. Remember that data science problems always target an audience and are meant to help a group of stakeholders solve a problem, so make sure that you explicitly describe your audience and why they would care about your problem.
   
### **Answer 1:** 
   **To provide the necessary information visualization to companies or different crop processing factories, I'll analyse the crop production datas of india in different region with Foursquare API.**


.

### **Question 2:**
   Describe the data that you will be using to solve the problem or execute your idea. Remember that you will need to use the Foursquare location data to solve the problem or execute your idea. You can absolutely use other datasets in combination with the Foursquare location data. So make sure that you provide adequate explanation and discussion, with examples, of the data that you will be using, even if it is only Foursquare location data.
   
### **Answer 2:**
   **For this project I will use Kaggle crop production dataset(https://www.kaggle.com/divyosmi2009/crop-production-in-india-statevise?select=crop_production.csv) which contains more than 2,00,000 data rows with the detailed information of crop production in India with respect to defferent region and year(**
   
| Year |%Data|
| ---- |----:|
| 1997 | Min |
| 2002 | 25% |
| 2006 | 50% |
| 2010 | 75% |
| 2015 | Max |

**) in combination with Forsquare location data.**


In [1]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests

#!conda install -c conda-forge beautifulsoup4 --yes
from bs4 import BeautifulSoup


import pandas as pd
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

### Reading Raw data from downloaded csv file

In [2]:
df = pd.read_csv('crop_production.csv')
sh = df.shape
print('This DataFrame Contains {} rows and {} columns.'.format(sh[0], sh[1]))
df.head()

This DataFrame Contains 246091 rows and 7 columns.


,State_Name,District_Name,Crop_Year,Season,Crop,Area,Production
0,Andaman and Nicobar Islands,NICOBARS,2000,Kharif,Arecanut,1254.0,2000.0
1,Andaman and Nicobar Islands,NICOBARS,2000,Kharif,Other Kharif pulses,2.0,1.0
2,Andaman and Nicobar Islands,NICOBARS,2000,Kharif,Rice,102.0,321.0
3,Andaman and Nicobar Islands,NICOBARS,2000,Whole Year,Banana,176.0,641.0
4,Andaman and Nicobar Islands,NICOBARS,2000,Whole Year,Cashewnut,720.0,165.0


### Creating required dataframe

In [3]:
df2 = pd.DataFrame(columns=['State_Name', 'District_Name', 'Crop_Year', 'Area', 'Production', 'Latitude', 'Longitude'])
df2

,State_Name,District_Name,Crop_Year,Area,Production,Latitude,Longitude


In [4]:
#define function to retrive location data of a given dist, state.
def get_loc(dist, state):
    address = '{}, {}'.format(dist, state)
    geolocator = Nominatim(user_agent="ny_explorer")
    location = geolocator.geocode(address)
    print(location)
    if location is None:
        location = geolocator.geocode(dist)
        print(location)
        if location is None:
            location = geolocator.geocode(state)
            print(location)
    return location 

### Filling Required Dataframe

In [5]:
area = 0
prod = 0
Crop_Year = df['Crop_Year'][0]
dist = df['District_Name'][0]
state = df['State_Name'][0]
location = get_loc(dist, state)

for tup in list(zip(df['District_Name'], df['State_Name'], df['Crop_Year'], df['Area'], df['Production'])):
    if (tup[0] != dist) or (tup[2] != Crop_Year):
        if tup[0] != dist:
            location = get_loc(dist, state)
            
        df2 = df2.append({'State_Name': state, 
                          'District_Name': dist, 
                          'Crop_Year': Crop_Year, 
                          'Area': area, 
                          'Production': prod, 
                          'Latitude': location.latitude, 
                          'Longitude': location.longitude}, ignore_index=True)
        area = 0
        prod = 0
        dist = tup[0]
        state = tup[1]
        Crop_Year = tup[2]

    try:
        area += tup[3]
        prod += tup[4]
    except:
        continue
else:
    df2 = df2.append({'State_Name': state, 
                      'District_Name': dist, 
                      'Crop_Year': Crop_Year, 
                      'Area': area, 
                      'Production': prod, 
                      'Latitude': location.latitude, 
                      'Longitude': location.longitude}, ignore_index=True)

df2

None
None
Andaman and Nicobar Islands, India
None
None
Andaman and Nicobar Islands, India
North and Middle Andaman, Andaman and Nicobar Islands, 744210, India
None
Taj Exotica Resort & Spa, Andamans, Path to Elephant Beach, Krishna nagar village, South Andaman, Andaman and Nicobar Islands, 744211, India
Anantapur, Andhra Pradesh, India
Chittoor, Andhra Pradesh, 517001, India
East Godavari, Andhra Pradesh, India
Guntur, Andhra Pradesh, 522001, India
Kadapa, YSR, Andhra Pradesh, 516001, India
Krishna, Andhra Pradesh, India
Kurnool, Andhra Pradesh, 518001, India
Prakasam, Andhra Pradesh, India
Sri Potti Sriramulu Nellore, Andhra Pradesh, India
Srikakulam, Andhra Pradesh, India
None
None
Andhra Pradesh, India
Vizianagaram, Andhra Pradesh, India
West Godavari, Andhra Pradesh, India
Anjaw, Arunachal Pradesh, 792104, India
Changlang, Changlang HQ, Changlang, Arunachal Pradesh, India
Upper Dibang Valley, Arunachal Pradesh, 792100, India
East Kameng, Arunachal Pradesh, India
East Siang, Upper S

,State_Name,District_Name,Crop_Year,Area,Production,Latitude,Longitude
0,Andaman and Nicobar Islands,NICOBARS,2000,20504.00,65103414.00,10.218834,92.577133
1,Andaman and Nicobar Islands,NICOBARS,2001,20306.00,64432688.00,10.218834,92.577133
2,Andaman and Nicobar Islands,NICOBARS,2002,21147.50,67494295.14,10.218834,92.577133
3,Andaman and Nicobar Islands,NICOBARS,2003,20822.44,68584031.23,10.218834,92.577133
4,Andaman and Nicobar Islands,NICOBARS,2004,20726.26,52382007.61,10.218834,92.577133
...,...,...,...,...,...,...,...
9834,West Bengal,PURULIA,2010,174840.00,308821.00,23.484541,88.556763
9835,West Bengal,PURULIA,2011,293253.00,789898.00,23.484541,88.556763
9836,West Bengal,PURULIA,2012,312076.00,780815.00,23.484541,88.556763
9837,West Bengal,PURULIA,2013,332815.00,806768.00,23.484541,88.556763


In [6]:
df2.head()

,State_Name,District_Name,Crop_Year,Area,Production,Latitude,Longitude
0,Andaman and Nicobar Islands,NICOBARS,2000,20504.00,65103414.00,10.218834,92.577133
1,Andaman and Nicobar Islands,NICOBARS,2001,20306.00,64432688.00,10.218834,92.577133
2,Andaman and Nicobar Islands,NICOBARS,2002,21147.50,67494295.14,10.218834,92.577133
3,Andaman and Nicobar Islands,NICOBARS,2003,20822.44,68584031.23,10.218834,92.577133
4,Andaman and Nicobar Islands,NICOBARS,2004,20726.26,52382007.61,10.218834,92.577133


In [7]:
df2.shape

(9839, 7)

In [8]:
len(set(zip(df2['Latitude'], df2['Longitude'], df2['State_Name'], df2['District_Name'])))

1295

### Creating map with Folium

In [9]:
address = 'India'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of India are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of India are 22.3511148, 78.6677428.


In [10]:
# create map of Toronto using latitude and longitude values
map_india = folium.Map(location=[latitude, longitude], zoom_start=4)

# add markers to map
for lat, lng, state, dist in set(zip(df2['Latitude'], df2['Longitude'], df2['State_Name'], df2['District_Name'])):
    label = '{}, {}'.format(dist, state)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_india)  
    
map_india